# Groupe5 - Apprentissage Artificiel

### FORDJOU KAMGANG Landry                            19M2366
### PANDJI TCHOUAKOUE Frank Manuel               19M2110
### AZESSIE NOGHIEWO Léonce                             19M2617
### EYENGA MINKONDA Laurentine Serena           19M2455

## TP1: Spam Email Classifier with KNN

### Importation des bibliothèques

In [1]:
import os
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

### 1- Chargement des donées

In [3]:
def load_data():
    print("Loading data...")
    #ham_files_location = os.listdir("enron2/ham")
    #spam_files_location = os.listdir("enron2/spam")
    spam_files_location = os.listdir("enron2/spam")
    ham_files_location = os.listdir("enron2/ham")
    data = []
    
    # Load ham email
    for file_path in ham_files_location:
        f = open("enron2/ham/" + file_path, "r")
        text = str(f.read())
        data.append([text, "ham"])
    
    # Load spam email
    for file_path in spam_files_location:
        f = open("enron2/spam/" + file_path, "r")
        text = str(f.read())
        data.append([text, "spam"])
        
    data = np.array(data)
    print("flag 1: loaded data")
    return data

In [4]:
mails = load_data()
print(len(mails))

Loading data...
flag 1: loaded data
5857


### 2- Pré-traitement des données

In [5]:
import nltk
from nltk.corpus import brown
#nltk.download('stopwords')
#nltk.download('brown')
#nltk.download()
#brown.words()


In [6]:
# Preprocessing data: noise removal
def preprocess_data(data):
    print("Preprocessing data...") 
    punc = string.punctuation           # Punctuation list
    sw = stopwords.words('english')     # Stopwords list
    for record in data:
        # Remove common punctuation and symbols
        for item in punc:
            record[0] = record[0].replace(item, "")
        # Lowercase all letters and remove stopwords 
            splittedWords = record[0].split()
            newText = ""
            for word in splittedWords:
                if word not in sw:
                    word = word.lower()
                    newText = newText + " " + word
            record[0] = newText
    print("flag 2: preprocessed data")        
    return data

In [75]:
preprocess_data(mails)

Preprocessing data...


KeyboardInterrupt: 

### Découpage du jeu de données en jeu d' apprentissage et jeu de test
Le découpage est fait dans les proportions 73%(apprentissage) - 27%(test)

In [7]:
# Splitting original dataset into training dataset and test dataset
def split_data(data):
    print("Splitting data...")
    features = data[:, 0]   # array containing all email text bodies
    labels = data[:, 1]     # array containing corresponding labels
    print(labels)
    training_data, test_data, training_labels, test_labels =\
        train_test_split(features, labels, test_size = 0.27, random_state = 42)
    
    print("flag 3: splitted data")
    return training_data, test_data, training_labels, test_labels

In [56]:
split_data(mails)

Splitting data...
['ham' 'ham' 'ham' ... 'spam' 'spam' 'spam']
flag 3: splitted data


(array([' subject new offrr want know ho liberalize w save 60 northward dlcatlons http www centr compassion alpan com successfull proven way rocking monumentalize ave money splash st prlces high qua homophone iity w grundyism orldwide shlpplng total confidenti despoil aiity 200 appeasement popular medlcatlons nice da upheave',
        ' subject enron default swaps hi vince got notes indeed useful one deutsche bank especially helpful suppose know stuff teach sorry delayed billing trouble getting bill excellent asistant taichi hoshino returned goldman tokyo able get anything else done lately try get something soon several energy people several companies credit risk exec ed course last month seems credit risk power risk go together days warm regards darrell fri 30 mar 2001 vince j kaminski enron com wrote darrell sending 2 technical notes enron default swaps hope useful shall read articles weekend curious find explanations satisfactory slow preparing number technical documents model revie

### Algorithme KNN

#### Fonction get_count()


In [8]:
def get_count(text):
    wordCounts = dict()
    for word in text.split():
        if word in wordCounts:
            wordCounts[word] += 1
        else:
            wordCounts[word] = 1
    
    return wordCounts

#### Fonction euclidean_difference()

In [9]:
def euclidean_difference(test_WordCounts, training_WordCounts):
    total = 0
    for word in test_WordCounts:
        if word in test_WordCounts and word in training_WordCounts:
            total += (test_WordCounts[word] - training_WordCounts[word])**2
            del training_WordCounts[word]
        else:
            total += test_WordCounts[word]**2
    return total**0.5

#### Fonction get_class()

In [14]:
def get_class(selected_Kvalues):
    spam_count = 0
    ham_count = 0
    for value in selected_Kvalues:
        if value[0] == "spam":
            spam_count += 1
        else:
            ham_count += 1
    if spam_count > ham_count:
        return "spam"
    else:
        return "ham"

#### Fonction knn_classifier()

In [26]:
def knn_classifier(training_data, training_labels, test_data, K,tsize):
    print("Running KNN Classifier...")
    result = []
    counter = 1
    # word counts for training email
    training_WordCounts = []
    for training_text in training_data:
        training_WordCounts.append(get_count(training_text))
    for test_text in test_data:
        similarity = [] # List of euclidean distances
        test_WordCounts = get_count(test_text) # word counts fortest email
        # Getting euclidean difference
        for index in range(len(training_data)):
            euclidean_diff =\
            euclidean_difference(test_WordCounts,training_WordCounts[index])
            similarity.append([training_labels[index],euclidean_diff])
        # Sort list in ascending order based on euclidean difference
        similarity = sorted(similarity, key = lambda i:i[1])
        # Select K nearest neighbours
        selected_Kvalues = []
        for i in range(K):
            selected_Kvalues.append(similarity[i])
        # Predicting the class of email
        result.append(get_class(selected_Kvalues))
    return result

### Fonction main()

In [27]:
def main(K):
    data = load_data()
    data = preprocess_data(data)
    training_data, test_data, training_labels, test_labels = split_data(data)
    tsize = len(data)
    result = knn_classifier(training_data, training_labels, test_data[:tsize], K, tsize)
    accuracy = accuracy_score(test_labels[:tsize], result)
    
    print("training data size\t: " + str(len(training_data)))
    print("test data size\t\t: " + str(len(test_data)))
    print("K value\t\t\t\t: " + str(K))
    print("Samples tested\t\t: " + str(tsize))
    print("% accuracy\t\t\t: " + str(accuracy * 100))
    print("Number correct\t\t: " + str(int(accuracy * tsize)))
    print("Number wrong\t\t: " + str(int((1 - accuracy) * tsize)))

In [28]:
main(11)

Loading data...
flag 1: loaded data
Preprocessing data...
flag 2: preprocessed data
Splitting data...
['ham' 'ham' 'ham' ... 'spam' 'spam' 'spam']
flag 3: splitted data
Running KNN Classifier...


KeyboardInterrupt: 